In [1]:
# 필요한 라이브러리 임포트
import pandas as pd

# 데이터 로드
visit_area_info_path = 'tn_visit_area_info_방문지정보_depthregion_cleaned.csv'
visit_area_info = pd.read_csv(visit_area_info_path)
# REVISIT_YN을 숫자로 변환 (Y -> 1, N -> 0)
visit_area_info['REVISIT_YN_NUMERIC'] = visit_area_info['REVISIT_YN'].map({'Y': 1, 'N': 0})

# TRAVEL_ID별 DGSTFN의 평균과 표준 편차 계산
travel_stats = visit_area_info.groupby('TRAVEL_ID')['DGSTFN'].agg(['mean', 'std']).reset_index()
travel_stats.rename(columns={'mean': 'Mean_Score', 'std': 'Std_Dev'}, inplace=True)

# 계산된 통계를 원래 데이터에 병합
visit_area_info = visit_area_info.merge(travel_stats, on='TRAVEL_ID', how='left')

# DGSTFN(만족도)을 사용자 평균과 표준 편차로 정규화
visit_area_info['Normalized_DGSTFN'] = (
    (visit_area_info['DGSTFN'] - visit_area_info['Mean_Score']) / visit_area_info['Std_Dev']
)
# NaN 처리 (표준 편차가 0인 경우 NaN 발생)
visit_area_info['Normalized_DGSTFN'] = visit_area_info['Normalized_DGSTFN'].fillna(0)

# 보정 함수 정의
def f(revisit_intention):
    """재방문의향 점수 보정 함수"""
    return 0.7 if revisit_intention <= 2 else 1.0

def g(revisit_intention, rcmdtn_intention):
    """추천의향과 재방문의향 간 불일치 보정 함수"""
    return 0.8 if (revisit_intention <= 2 and rcmdtn_intention > 3) else 1.0

# 가중치 설정
weights = {
    'REVISIT_YN_NUMERIC': 0.25,  # 재방문 여부 가중치
    'Normalized_DGSTFN': 0.40,  # 정규화된 만족도 점수 가중치
    'REVISIT_INTENTION': 0.20,  # 재방문의향 가중치
    'RCMDTN_INTENTION': 0.15    # 추천의향 가중치
}

# 최종 점수 계산
visit_area_info['Calculated_Final_Score'] = (
    weights['REVISIT_YN_NUMERIC'] * visit_area_info['REVISIT_YN_NUMERIC'] +  # 재방문 여부 반영
    weights['Normalized_DGSTFN'] * visit_area_info['Normalized_DGSTFN'] +  # 정규화된 만족도 점수 반영
    weights['REVISIT_INTENTION'] * visit_area_info['REVISIT_INTENTION'].apply(f) +  # 재방문의향 보정
    weights['RCMDTN_INTENTION'] * visit_area_info.apply(
        lambda x: x['RCMDTN_INTENTION'] * g(x['REVISIT_INTENTION'], x['RCMDTN_INTENTION']), axis=1
    )
).round(2)

# 결과를 CSV 파일로 저장
output_path = 'tn_visit_area_info_방문지정보_depth_cleaned_score.csv'
visit_area_info.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"결과 파일이 생성되었습니다: {output_path}")


결과 파일이 생성되었습니다: tn_visit_area_info_방문지정보_depth_cleaned_score.csv


In [2]:



#CATEGORY 랜덤으로 뽑은거임
#CATEGORY 랜덤으로 뽑은거임
#CATEGORY 랜덤으로 뽑은거임
#CATEGORY 랜덤으로 뽑은거임 place api로 대체예정
import random

# CATEGORY 컬럼 생성 및 값 추가
categories = ['음식점', '액티비티', '숙박']  # CATEGORY 값 목록
visit_area_info['CATEGORY'] = visit_area_info.apply(
    lambda _: random.choice(categories), axis=1
)  # 각 행에 랜덤 값 할당

# 결과 확인
print("CATEGORY 컬럼이 생성되었습니다. 일부 데이터:")



CATEGORY 컬럼이 생성되었습니다. 일부 데이터:


랭킹산출

In [ ]:
import pandas as pd
import numpy as np

# 특정 'CATEGORY' 값을 기준으로 상위 10개를 추출하며, CSV로 저장
def process_top_10_by_category_v5(region_name, category, output_file_prefix='top_10_results'):
    # 데이터 필터링
    filtered_data = visit_area_info[
        (visit_area_info['SGG_NM'] == region_name) &
        (visit_area_info['CATEGORY'] == category)
    ]

    if not filtered_data.empty:
        # 상위 10개 데이터 필터링
        top_10_data = filtered_data.nlargest(10, 'Calculated_Final_Score').copy()

        # 점수 비율 계산
        top_10_data['Score_Percentage'] = (
            top_10_data['Calculated_Final_Score'] / top_10_data['Calculated_Final_Score'].sum() * 100
        ).fillna(0)

        # 점수 기반 가중치 생성
        top_10_data['Weight'] = top_10_data['Score_Percentage'] / top_10_data['Score_Percentage'].sum()
        top_10_data['Weight'] = top_10_data['Weight'].fillna(1 / len(top_10_data))  # 가중치가 0인 경우 기본값 설정

        # 확률 분포가 유효한지 확인
        if len(top_10_data) > 1 and top_10_data['Weight'].sum() > 0:
            # 가중치 정규화 (합이 정확히 1이 되도록)
            top_10_data['Weight'] = top_10_data['Weight'] / top_10_data['Weight'].sum()

            # 순위 배정: 랜덤하게 배정
            rank_size = len(top_10_data)
            top_10_data['Rank'] = np.random.choice(
                range(1, rank_size + 1), size=len(top_10_data), replace=False, p=top_10_data['Weight']
            )

            # 결과 컬럼 처리
            top_10_data['POI_or_VISIT_AREA'] = top_10_data['POI_NM'].combine_first(top_10_data['VISIT_AREA_NM'])
            top_10_data['ADDR'] = top_10_data['ROAD_NM_ADDR'].combine_first(top_10_data['LOTNO_ADDR'])

            # 결과 정렬
            top_10_data_sorted = top_10_data.sort_values(by='Rank')

            # CSV 저장 컬럼 순서 지정
            save_columns = [
                'CATEGORY', 'Rank', 'TRAVEL_ID', 'POI_or_VISIT_AREA', 'ADDR',
                'Calculated_Final_Score', 'Score_Percentage', 'Weight'
            ]

            # CSV 파일 이름 설정
            output_file = f"{output_file_prefix}_{category}.csv"

            # 데이터 저장
            top_10_data_sorted.to_csv(output_file, columns=save_columns, index=False, encoding='utf-8-sig')
            print(f"결과가 {category}에 대해 CSV 파일로 저장되었습니다: {output_file}")

            # RANK, CATEGORY, POI_or_VISIT_AREA, ADDR만 출력
            print(f"Top 10 Results for Category: {category}")
            print(top_10_data_sorted[['CATEGORY', 'Rank', 'POI_or_VISIT_AREA', 'ADDR']])
        else:
            print(f"'{region_name}'에 해당하는 데이터 또는 CATEGORY '{category}'가 없습니다.")
    else:
        print(f"'{region_name}'에 해당하는 데이터 또는 CATEGORY '{category}'가 없습니다.")

# 예시 실행: '수원시 장안구'에서 '음식점' 상위 10개 추출 및 저장
process_top_10_by_category_v5('수원시 장안구', '음식점')


결과가 음식점에 대해 CSV 파일로 저장되었습니다: top_10_results_음식점.csv
Top 10 Results for Category: 음식점
      CATEGORY  Rank    POI_or_VISIT_AREA                       ADDR
654        음식점     1             수원KT위즈파크        경기 수원시 장안구 경수대로 893
16362      음식점     2                동북교자관      경기 수원시 장안구 팔달로247번길 4
10266      음식점     3         화이트스노우 성균관대점        경기 수원시 장안구 서부로 2132
10265      음식점     4                북수원온천        경기 수원시 장안구 서부로 2139
7109       음식점     5             수원KT위즈파크        경기 수원시 장안구 경수대로 893
2021       음식점     6           풍어생선구이정식입구  경기 수원시 장안구 천천로210번길 17-12
64085      음식점     7             우리동네스시신조    경기 수원시 장안구 만석로19번길 25-5
14490      음식점     8  더카페 라운지앤페르케노 수원성대역점    경기 수원시 장안구 서부로2126번길 10
5645       음식점     9      스타벅스 수원정자사거리DT점         경기 수원시 장안구 수성로 291
4420       음식점    10                   일랑  경기 수원시 장안구 경수대로927번길 12-4


: 